In [ ]:
from django_for_jupyter import init_django

init_django()

In [ ]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from django.db.models import F, Sum, Count, Q

In [ ]:
from precios.models import SiteURLResults, Articulos, Unifica, Marcas


In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
from selenium.webdriver.chrome.service import Service as ChromiumService
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.core.utils import ChromeType

import chromedriver_autoinstaller
try:
    chromedriver_autoinstaller.install() 
except:
    print('Sin conexcion')


def compare_and_find_paths_with_selenium2(url1, url2):
    # Configuramos el navegador (en este caso, Chrome)
    options = webdriver.ChromeOptions()
    options.add_argument("--enable-javascript")
    # options.add_argument("--headless")  # Para ejecutar el navegador en segundo plano, sin interfaz gráfica
    # driver = webdriver.Chrome(options=options)
    driver = webdriver.Chrome(service=ChromiumService(ChromeDriverManager(chrome_type=ChromeType.CHROMIUM).install()),options=options)

    # Cargamos la primera URL
    driver.get(url1)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))

    # Obtenemos el contenido de la primera página
    content1 = driver.page_source

    # Cargamos la segunda URL
    driver.get(url2)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))

    # Obtenemos el contenido de la segunda página
    content2 = driver.page_source

    

    # Procesamos los contenidos y encontramos las diferencias
    unique_content_url1 = set(BeautifulSoup(content1, 'html.parser').stripped_strings)
    unique_content_url2 = set(BeautifulSoup(content2, 'html.parser').stripped_strings)

    differences_url1 = unique_content_url1 - unique_content_url2
    differences_url2 = unique_content_url2 - unique_content_url1

    paths_url1 = {}
    paths_url2 = {}

    # Resto del código sin cambios
    # Buscamos los paths del DOM de las diferencias
    for content in differences_url1:
        # elements = driver.find_elements_by_xpath(f"//*[contains(text(), '{content}')]")
        elements = driver.find_element(By.XPATH,(f"//*[contains(text(), '{content}')]"))
        # elements = driver.find_element(By.XPATH,("//meta[@name='description']"))
        for element in elements:
            path = driver.execute_script(
                "alert();"
                "function absoluteXPath(element) {"
                "    var comp, comps = [];"
                "    var parent = null;"
                "    var xpath = '';"
                "    var getPos = function(element) {"
                "        var position = 1, curNode;"
                "        if (element.nodeType == Node.ATTRIBUTE_NODE) {"
                "            return null;"
                "        }"
                "        for (curNode = element.previousSibling; curNode; curNode = curNode.previousSibling) {"
                "            if (curNode.nodeName == element.nodeName) {"
                "                ++position;"
                "            }"
                "        }"
                "        return position;"
                "    };"
                ""
                "    if (element instanceof Document) {"
                "        return '/';"
                "    }"
                ""
                "    for (; element && !(element instanceof Document); element = element.nodeType == Node.ATTRIBUTE_NODE ? element.ownerElement : element.parentNode) {"
                "        comp = comps[comps.length] = {};"
                "        switch (element.nodeType) {"
                "        case Node.TEXT_NODE:"
                "            comp.name = 'text()';"
                "            break;"
                "        case Node.ATTRIBUTE_NODE:"
                "            comp.name = '@' + element.nodeName;"
                "            break;"
                "        case Node.PROCESSING_INSTRUCTION_NODE:"
                "            comp.name = 'processing-instruction()';"
                "            break;"
                "        case Node.COMMENT_NODE:"
                "            comp.name = 'comment()';"
                "            break;"
                "        case Node.ELEMENT_NODE:"
                "            comp.name = element.nodeName;"
                "            break;"
                "        }"
                "        comp.position = getPos(element);"
                "    }"
                ""
                "    for (var i = comps.length - 1; i >= 0; i--) {"
                "        comp = comps[i];"
                "        xpath += '/' + comp.name.toLowerCase();"
                "        if (comp.position !== null) {"
                "            xpath += '[' + comp.position + ']';"
                "        }"
                "    }"
                ""
                "    return xpath;"
                "};"
                "return absoluteXPath(arguments[0]);", element
            )
            paths_url1[content] = path

    for content in differences_url2:
        elements = driver.find_elements_by_xpath(f"//*[contains(text(), '{content}')]")
        for element in elements:
            path = driver.execute_script(
                "function absoluteXPath(element) {"
                "    var comp, comps = [];"
                "    var parent = null;"
                "    var xpath = '';"
                "    var getPos = function(element) {"
                "        var position = 1, curNode;"
                "        if (element.nodeType == Node.ATTRIBUTE_NODE) {"
                "            return null;"
                "        }"
                "        for (curNode = element.previousSibling; curNode; curNode = curNode.previousSibling) {"
                "            if (curNode.nodeName == element.nodeName) {"
                "                ++position;"
                "            }"
                "        }"
                "        return position;"
                "    };"
                ""
                "    if (element instanceof Document) {"
                "        return '/';"
                "    }"
                ""
                "    for (; element && !(element instanceof Document); element = element.nodeType == Node.ATTRIBUTE_NODE ? element.ownerElement : element.parentNode) {"
                "        comp = comps[comps.length] = {};"
                "        switch (element.nodeType) {"
                "        case Node.TEXT_NODE:"
                "            comp.name = 'text()';"
                "            break;"
                "        case Node.ATTRIBUTE_NODE:"
                "            comp.name = '@' + element.nodeName;"
                "            break;"
                "        case Node.PROCESSING_INSTRUCTION_NODE:"
                "            comp.name = 'processing-instruction()';"
                "            break;"
                "        case Node.COMMENT_NODE:"
                "            comp.name = 'comment()';"
                "            break;"
                "        case Node.ELEMENT_NODE:"
                "            comp.name = element.nodeName;"
                "            break;"
                "        }"
                "        comp.position = getPos(element);"
                "    }"
                ""
                "    for (var i = comps.length - 1; i >= 0; i--) {"
                "        comp = comps[i];"
                "        xpath += '/' + comp.name.toLowerCase();"
                "        if (comp.position !== null) {"
                "            xpath += '[' + comp.position + ']';"
                "        }"
                "    }"
                ""
                "    return xpath;"
                "};"
                "return absoluteXPath(arguments[0]);", element
            )
            paths_url2[content] = path
    driver.quit()

    return paths_url1, paths_url2



def compare_and_find_paths_with_selenium(url1, url2):
    # Configuramos el navegador (en este caso, Chrome)
    options = webdriver.ChromeOptions()
    # options.add_argument("--headless")  # Para ejecutar el navegador en segundo plano, sin interfaz gráfica
    options.add_argument("--enable-javascript")
    driver = webdriver.Chrome(options=options)

    # Cargamos la primera URL
    driver.get(url1)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.TAG_NAME, 'h2')))
    time.sleep(5)

    # Obtenemos el contenido de la primera página
    content1 = driver.page_source

    # Cargamos la segunda URL
    driver.get(url2)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.TAG_NAME, 'h2')))
    time.sleep(5)

    # Obtenemos el contenido de la segunda página
    content2 = driver.page_source

    

    # Procesamos los contenidos y encontramos las diferencias
    unique_content_url1 = set(BeautifulSoup(content1, 'html.parser').stripped_strings)
    unique_content_url2 = set(BeautifulSoup(content2, 'html.parser').stripped_strings)

    differences_url1 = unique_content_url1 - unique_content_url2
    differences_url2 = unique_content_url2 - unique_content_url1

    paths_url1 = {}
    paths_url2 = {}

    # Buscamos los paths del DOM de las diferencias
    for content in differences_url1:
        # elements = driver.find_elements_by_xpath(f"//*[contains(text(), '{content}')]")
        elements = driver.find_element(By.XPATH,(f"//*[contains(text(), '{content}')]"))
        # elements = driver.find_element(By.XPATH,("//meta[@name='description']"))
        for element in elements:
            path = driver.execute_script(
                "alert();"
                "function absoluteXPath(element) {"
                "    var comp, comps = [];"
                "    var parent = null;"
                "    var xpath = '';"
                "    var getPos = function(element) {"
                "        var position = 1, curNode;"
                "        if (element.nodeType == Node.ATTRIBUTE_NODE) {"
                "            return null;"
                "        }"
                "        for (curNode = element.previousSibling; curNode; curNode = curNode.previousSibling) {"
                "            if (curNode.nodeName == element.nodeName) {"
                "                ++position;"
                "            }"
                "        }"
                "        return position;"
                "    };"
                ""
                "    if (element instanceof Document) {"
                "        return '/';"
                "    }"
                ""
                "    for (; element && !(element instanceof Document); element = element.nodeType == Node.ATTRIBUTE_NODE ? element.ownerElement : element.parentNode) {"
                "        comp = comps[comps.length] = {};"
                "        switch (element.nodeType) {"
                "        case Node.TEXT_NODE:"
                "            comp.name = 'text()';"
                "            break;"
                "        case Node.ATTRIBUTE_NODE:"
                "            comp.name = '@' + element.nodeName;"
                "            break;"
                "        case Node.PROCESSING_INSTRUCTION_NODE:"
                "            comp.name = 'processing-instruction()';"
                "            break;"
                "        case Node.COMMENT_NODE:"
                "            comp.name = 'comment()';"
                "            break;"
                "        case Node.ELEMENT_NODE:"
                "            comp.name = element.nodeName;"
                "            break;"
                "        }"
                "        comp.position = getPos(element);"
                "    }"
                ""
                "    for (var i = comps.length - 1; i >= 0; i--) {"
                "        comp = comps[i];"
                "        xpath += '/' + comp.name.toLowerCase();"
                "        if (comp.position !== null) {"
                "            xpath += '[' + comp.position + ']';"
                "        }"
                "    }"
                ""
                "    return xpath;"
                "};"
                "return absoluteXPath(arguments[0]);", element
            )
            paths_url1[content] = path

    for content in differences_url2:
        elements = driver.find_elements_by_xpath(f"//*[contains(text(), '{content}')]")
        for element in elements:
            path = driver.execute_script(
                "function absoluteXPath(element) {"
                "    var comp, comps = [];"
                "    var parent = null;"
                "    var xpath = '';"
                "    var getPos = function(element) {"
                "        var position = 1, curNode;"
                "        if (element.nodeType == Node.ATTRIBUTE_NODE) {"
                "            return null;"
                "        }"
                "        for (curNode = element.previousSibling; curNode; curNode = curNode.previousSibling) {"
                "            if (curNode.nodeName == element.nodeName) {"
                "                ++position;"
                "            }"
                "        }"
                "        return position;"
                "    };"
                ""
                "    if (element instanceof Document) {"
                "        return '/';"
                "    }"
                ""
                "    for (; element && !(element instanceof Document); element = element.nodeType == Node.ATTRIBUTE_NODE ? element.ownerElement : element.parentNode) {"
                "        comp = comps[comps.length] = {};"
                "        switch (element.nodeType) {"
                "        case Node.TEXT_NODE:"
                "            comp.name = 'text()';"
                "            break;"
                "        case Node.ATTRIBUTE_NODE:"
                "            comp.name = '@' + element.nodeName;"
                "            break;"
                "        case Node.PROCESSING_INSTRUCTION_NODE:"
                "            comp.name = 'processing-instruction()';"
                "            break;"
                "        case Node.COMMENT_NODE:"
                "            comp.name = 'comment()';"
                "            break;"
                "        case Node.ELEMENT_NODE:"
                "            comp.name = element.nodeName;"
                "            break;"
                "        }"
                "        comp.position = getPos(element);"
                "    }"
                ""
                "    for (var i = comps.length - 1; i >= 0; i--) {"
                "        comp = comps[i];"
                "        xpath += '/' + comp.name.toLowerCase();"
                "        if (comp.position !== null) {"
                "            xpath += '[' + comp.position + ']';"
                "        }"
                "    }"
                ""
                "    return xpath;"
                "};"
                "return absoluteXPath(arguments[0]);", element
            )
            paths_url2[content] = path
            
    driver.quit()
    return paths_url1, paths_url2

if __name__ == "__main__":
    url1 = "https://www.jumbo.cl/estufa-fibra-de-carbono-thomas-th-rfc50-1952567/p"
    url2 = "https://www.jumbo.cl/exprimidor-de-citricos-fpstju104-052/p"

    paths_url1, paths_url2 = compare_and_find_paths_with_selenium2(url1, url2)

    print("Diferencias en la primera URL:")
    for content, path in paths_url1.items():
        print(f"- Texto: {content}, Path: {path}")

    print("\nDiferencias en la segunda URL:")
    for content, path in paths_url2.items():
        print(f"- Texto: {content}, Path: {path}")


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[contains(text(), '1720 KCAL/HR')]"}
  (Session info: chrome=114.0.5735.198)
Stacktrace:
#0 0x564a62c344e3 <unknown>
#1 0x564a62963c76 <unknown>
#2 0x564a6299fc96 <unknown>
#3 0x564a6299fdc1 <unknown>
#4 0x564a629d97f4 <unknown>
#5 0x564a629bf03d <unknown>
#6 0x564a629d730e <unknown>
#7 0x564a629bede3 <unknown>
#8 0x564a629942dd <unknown>
#9 0x564a6299534e <unknown>
#10 0x564a62bf43e4 <unknown>
#11 0x564a62bf83d7 <unknown>
#12 0x564a62c02b20 <unknown>
#13 0x564a62bf9023 <unknown>
#14 0x564a62bc71aa <unknown>
#15 0x564a62c1d6b8 <unknown>
#16 0x564a62c1d847 <unknown>
#17 0x564a62c2d243 <unknown>
#18 0x7f10f88d6b43 <unknown>


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# El resto del código es el mismo

def compare_and_find_paths_with_selenium(url1, url2):
    # Configuramos el navegador (en este caso, Chrome)
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Para ejecutar el navegador en segundo plano, sin interfaz gráfica
    driver = webdriver.Chrome(options=options)

    # Cargamos la primera URL
    driver.get(url1)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))

    # Obtenemos el contenido de la primera página
    content1 = driver.page_source

    # Cargamos la segunda URL
    driver.get(url2)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))

    # Obtenemos el contenido de la segunda página
    content2 = driver.page_source

    driver.quit()

    # Procesamos los contenidos y encontramos las diferencias
    unique_content_url1 = set(BeautifulSoup(content1, 'html.parser').stripped_strings)
    unique_content_url2 = set(BeautifulSoup(content2, 'html.parser').stripped_strings)

    differences_url1 = unique_content_url1 - unique_content_url2
    differences_url2 = unique_content_url2 - unique_content_url1

    paths_url1 = {}
    paths_url2 = {}

    # Resto del código sin cambios

    return paths_url1, paths_url2

# Resto del código sin cambios


In [ ]:
import requests
from bs4 import BeautifulSoup

def find_element_path(soup, element, path=''):
    if not element.parent:
        return path

    siblings = element.parent.find_all(element.name, recursive=False)
    index = siblings.index(element) + 1  # +1 porque los índices comienzan en 0
    path = f"{element.name}[{index}] > {path}"
    return find_element_path(soup, element.parent, path)

def compare_and_find_paths(url1, url2):
    response1 = requests.get(url1)
    soup1 = BeautifulSoup(response1.content, 'html.parser')

    response2 = requests.get(url2)
    soup2 = BeautifulSoup(response2.content, 'html.parser')

    unique_content_url1 = set(soup1.stripped_strings)
    unique_content_url2 = set(soup2.stripped_strings)

    differences_url1 = unique_content_url1 - unique_content_url2
    differences_url2 = unique_content_url2 - unique_content_url1

    paths_url1 = {}
    paths_url2 = {}

    for content in differences_url1:
        elements = soup1.find_all(text=content)
        for element in elements:
            path = find_element_path(soup1, element)
            paths_url1[content] = path

    for content in differences_url2:
        elements = soup2.find_all(text=content)
        for element in elements:
            path = find_element_path(soup2, element)
            paths_url2[content] = path

    return paths_url1, paths_url2

if __name__ == "__main__":
    url1 = "https://www.jumbo.cl/estufa-fibra-de-carbono-thomas-th-rfc50-1952567/p"
    url2 = "https://www.jumbo.cl/exprimidor-de-citricos-fpstju104-052/p"

    paths_url1, paths_url2 = compare_and_find_paths(url1, url2)

    print("Diferencias en la primera URL:")
    for content, path in paths_url1.items():
        print(f"- Texto: {content}, Path: {path}")

    print("\nDiferencias en la segunda URL:")
    for content, path in paths_url2.items():
        print(f"- Texto: {content}, Path: {path}")


In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

def find_element_path(soup, element, path=''):
    if not element.parent:
        return path

    siblings = element.parent.find_all(element.name, recursive=False)
    index = siblings.index(element) + 1
    path = f"{element.name}[{index}] > {path}"
    return find_element_path(soup, element.parent, path)

def compare_and_find_paths_with_selenium(url1, url2):
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)

    driver.get(url1)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.TAG_NAME, 'h2')))
    time.sleep(5)
    content1 = driver.page_source
    

    driver.get(url2)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.TAG_NAME, 'h2')))
    time.sleep(5)
    content2 = driver.page_source

    driver.quit()

    # Procesamos los contenidos y encontramos las diferencias
    soup1 = BeautifulSoup(content1, 'html.parser')
    soup2 = BeautifulSoup(content2, 'html.parser')
    
    unique_content_url1 = set([element.text.strip() for element in soup1.find_all('h2')])
    unique_content_url2 = set([element.text.strip() for element in soup2.find_all('h2')])

    differences_url1 = unique_content_url1 - unique_content_url2
    differences_url2 = unique_content_url2 - unique_content_url1

    paths_url1 = {}
    paths_url2 = {}

    for content in differences_url1:
        elements = soup1.find_all(text=content)
        for element in elements:
            path = find_element_path(soup1, element)
            paths_url1[content] = path

    for content in differences_url2:
        elements = soup2.find_all(text=content)
        for element in elements:
            path = find_element_path(soup2, element)
            paths_url2[content] = path

    return paths_url1, paths_url2

if __name__ == "__main__":
    url1 = "https://www.jumbo.cl/estufa-fibra-de-carbono-thomas-th-rfc50-1952567/p"
    url2 = "https://www.jumbo.cl/exprimidor-de-citricos-fpstju104-052/p"

    paths_url1, paths_url2 = compare_and_find_paths_with_selenium(url1, url2)

    print("Diferencias en la primera URL:")
    for content, path in paths_url1.items():
        print(f"- Texto: {content}, Path: {path}")

    print("\nDiferencias en la segunda URL:")
    for content, path in paths_url2.items():
        print(f"- Texto: {content}, Path: {path}")


In [ ]:
import requests
from bs4 import BeautifulSoup

def get_page_content(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup.get_text()

def compare_pages(url1, url2):
    content1 = get_page_content(url1)
    content2 = get_page_content(url2)

    unique_content_url1 = set(content1.strip().split())
    unique_content_url2 = set(content2.strip().split())

    # unique_content_url1 = set(content1.strip())
    # unique_content_url2 = set(content2.strip())

    differences_url1 = unique_content_url1 - unique_content_url2
    differences_url2 = unique_content_url2 - unique_content_url1

    return differences_url1, differences_url2

if __name__ == "__main__":
    url1 = "https://www.jumbo.cl/estufa-fibra-de-carbono-thomas-th-rfc50-1952567/p"
    url2 = "https://www.jumbo.cl/exprimidor-de-citricos-fpstju104-052/p"

    differences_url1, differences_url2 = compare_pages(url1, url2)

    print("Diferencias en la primera URL:")
    print(differences_url1)

    print("\nDiferencias en la segunda URL:")
    print(differences_url2)


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

def get_page_content_with_selenium(url):
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
    content = driver.page_source
    driver.quit()
    return content

def compare_pages_with_selenium(url1, url2):
    content1 = get_page_content_with_selenium(url1)
    content2 = get_page_content_with_selenium(url2)

    soup1 = BeautifulSoup(content1, 'html.parser')
    soup2 = BeautifulSoup(content2, 'html.parser')

    unique_content_url1 = set(soup1.get_text().strip().split())
    unique_content_url2 = set(soup2.get_text().strip().split())

    differences_url1 = unique_content_url1 - unique_content_url2
    differences_url2 = unique_content_url2 - unique_content_url1

    return differences_url1, differences_url2

if __name__ == "__main__":
    url1 = "https://www.jumbo.cl/estufa-fibra-de-carbono-thomas-th-rfc50-1952567/p"
    url2 = "https://www.jumbo.cl/exprimidor-de-citricos-fpstju104-052/p"


    differences_url1, differences_url2 = compare_pages_with_selenium(url1, url2)

    print("Diferencias en la primera URL:")
    print(differences_url1)

    print("\nDiferencias en la segunda URL:")
    print(differences_url2)


In [ ]:
import time
from selenium import webdriver
from bs4 import BeautifulSoup

def get_visible_text(driver):
    return driver.execute_script("return document.documentElement.innerText;")
    # return driver.execute_script("return document.documentElement.textContent;")

def compare_pages_with_selenium(url1, url2):
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)

    driver.get(url1)
    time.sleep(5)  # Pausa de 5 segundos para esperar cualquier carga dinámica adicional
    content1 = get_visible_text(driver)

    driver.get(url2)
    time.sleep(5)  # Pausa de 5 segundos para esperar cualquier carga dinámica adicional
    content2 = get_visible_text(driver)

    driver.quit()

    unique_content_url1 = set(content1.strip().split(' '))
    unique_content_url2 = set(content2.strip().split(' '))

    # unique_content_url1 = set(content1.strip())
    # unique_content_url2 = set(content2.strip())

    differences_url1 = unique_content_url1 - unique_content_url2
    differences_url2 = unique_content_url2 - unique_content_url1

    return differences_url1, differences_url2

if __name__ == "__main__":
    url1 = "https://www.jumbo.cl/estufa-fibra-de-carbono-thomas-th-rfc50-1952567/p"
    url2 = "https://www.jumbo.cl/exprimidor-de-citricos-fpstju104-052/p"

    differences_url1, differences_url2 = compare_pages_with_selenium(url1, url2)

    print("Diferencias en la primera URL:")
    print(differences_url1)

    print("\nDiferencias en la segunda URL:")
    print(differences_url2)
